# Initialisation 

In [1]:
#Import data 
from tqdm import tqdm
import numpy as np
import time
from sklearn.decomposition import PCA
from tqdm import tqdm
from random import randrange
import sklearn
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.sparse import diags
from scipy.stats import multivariate_normal
from scipy.ndimage.morphology import distance_transform_edt as bwdist
import scipy.ndimage as nd
from scipy.interpolate import RegularGridInterpolator
import tensorflow as tf
import keras
import scipy.stats as ss 
from keras.constraints import Constraint
from keras import backend as K
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
from google.colab import drive
#drive.mount('/content/drive')
#from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Mounted at /content/drive
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2417112855465611824
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15703311680
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1922738279955809365
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


# Parameters 

In [2]:
DirSAVE                     = '/'
flagTrWMissingData          = 0  # Training phase with or without missing data
flagloadOIData              = 0    # load OI: work on rough variable or anomaly
lname_cov                   = ["ssh_mod"]
lid_cov                     = ["OI"]
size_tw                     = 1   # Length of the 4th dimension          
Wsquare                     = 4     # half-width of holes
Nsquare                     = 3     # number of holes
DimAE                       = 20  # Dimension of the latent space
flagLoadModel               = 0     # load pre-defined AE model or not
sigNoise                    = 1e-1
flagTrOuputWOMissingData    = 1
stdMask                     = 0.
dropout                     = 0.03 #[0.6,0.7,0.8]0.03
wl2                         = 0.0000
batch_size                  = 16 
NbEpoc                      = 6
Niter                       = 6*3
thrMisData                  = 0.15 #noise
N_cov                       = 0
flagUseMaskinEncoder        = 0

def createGlobParams(params):
    return dict(((k, eval(k)) for k in params))
list_globParams=[
    'flagTrOuputWOMissingData','flagTrWMissingData',\
    'flagloadOIData','size_tw','Wsquare',\
    'Nsquare','DimAE','flagLoadModel',\
    'sigNoise',\
    'stdMask',\
    'dropout','wl2','batch_size',\
    'NbEpoc','Niter','DirSAVE','N_cov']
globParams = createGlobParams(list_globParams)

# Import data

In [3]:
dataset= np.load("*/Dataset_64_ZOI.npy",allow_pickle='TRUE').item()
dataH=dataset['CSED_Hourly']
mask=dataset['Cloud_Daily']
lat_grid=dataset['Lat_ZOI']
lon_grid=dataset['Lon_ZOI']
#corr lalpaciens var exp des norm
# =============================================================================
# Creation of the test and training dataset
# =============================================================================
def prepdata(xH,mask,N_Catalog=0):
    if N_Catalog!=0:
        test=np.zeros(mask[-365:].shape)
        for i in range(0,len(test)):
            test[i]=xH[26304+i*24+12].reshape(test.shape[1],test.shape[2])

        train=np.zeros((N_Catalog*365,xH.shape[1],xH.shape[2]))

        for i in range(N_Catalog):
            for j in range(len(test)):
                h=randrange(0,24)
                train[i*365+j]=xH[i*365+j*24+h]
    else:
        xD=np.empty((xH.shape[0]//24,xH.shape[1],xH.shape[2]))
        for i in range(len(xD)):
            xD[i]=xH[12+i*24]
    mask_train = mask[:-100]
    mask_pred  = mask[-100:]
    x_train    = xD[:len(mask_train)]
    y_pred     = xD[len(mask_train):len(mask_train)+len(mask_pred)]
    return mask_train,mask_pred,x_train,y_pred


mask[np.where(mask==0)]=1;mask[np.where(np.isnan(mask))]=0 # O for missing data
mask_train,mask_pred,x_train,y_pred = prepdata(dataH,mask)

x_train,x_test,mask_train,mask_test=sklearn.model_selection.train_test_split(x_train,mask_train,test_size=0.33)
# list of test dates
indN_Tt = np.arange(len(x_train),len(x_train)+len(x_test))
indN_Tr = np.arange(len(x_train))
lday_test=[ datetime.strftime(datetime.strptime("2001-1-01",'%Y-%m-%d')\
                      + timedelta(days=np.float64(i)),"%Y-%m-%d") for i in indN_Tt ]
indLat     = np.arange(0,64)
indLon     = np.arange(0,64)      
print(indLat.shape,indLon.shape,indN_Tr.shape,indN_Tt.shape)
print(x_train.shape,x_test.shape,y_pred.shape)

(64,) (64,) (895,) (441,)
(895, 64, 64) (441, 64, 64) (100, 64, 64)


# Normalize

In [5]:
gt_train= x_train
gt_test = x_test
gt_pred = np.copy(y_pred)
err_train = np.random.normal(0,thrMisData*np.nanvar(x_train),(x_train.shape))
err_test = np.random.normal(0,thrMisData*np.nanvar(x_test),(x_test.shape))
err_pred = np.random.normal(0,thrMisData*np.nanvar(y_pred),(y_pred.shape))

stdsc=sklearn.preprocessing.StandardScaler()
stdsc.fit(y_pred.reshape(y_pred.shape[0],-1))
mean_Tr = stdsc.mean_
std_Tr = stdsc.var_

gt_train        = gt_train.reshape(x_train.shape[0],-1)-mean_Tr
x_train         = x_train.reshape(x_train.shape[0],-1)-mean_Tr
gt_test         = gt_test.reshape(len(x_test),-1)-mean_Tr                       
x_test          = x_test.reshape(len(x_test),-1)-mean_Tr
y_pred          = y_pred.reshape(len(y_pred),-1)-mean_Tr

if flagTrWMissingData == 1 :
    x_train_missing=x_train.reshape(x_train.shape[0],indLat.shape[0],indLon.shape[0])*mask_train
else :
    mask_train[:]=1
    x_train_missing=x_train.reshape(x_train.shape[0],indLat.shape[0],indLon.shape[0])*mask_train

x_test_missing=x_test.reshape(x_test.shape[0],indLat.shape[0],indLon.shape[0])*mask_test
y_pred_missing=y_pred.reshape(y_pred.shape[0],indLat.shape[0],indLon.shape[0])*mask_pred

#Dimensionning                           
gt_train        = gt_train.reshape(x_train.shape[0],indLat.shape[0],indLon.shape[0],1)
x_train         = x_train.reshape(x_train.shape[0],indLat.shape[0],indLon.shape[0],1)
x_train_missing = x_train_missing.reshape(x_train_missing.shape[0],indLat.shape[0],indLon.shape[0],1)
mask_train      = mask_train.reshape(x_train.shape[0],indLat.shape[0],indLon.shape[0],1)

gt_test         = gt_test.reshape(indN_Tt.shape[0],indLat.shape[0],indLon.shape[0],1)
x_test          = x_test.reshape(indN_Tt.shape[0],indLat.shape[0],indLon.shape[0],1)
x_test_missing  = x_test_missing.reshape(indN_Tt.shape[0],indLat.shape[0],indLon.shape[0],1)   
mask_test       = mask_test.reshape(indN_Tt.shape[0],indLat.shape[0],indLon.shape[0],1)

y_pred          = y_pred.reshape(y_pred.shape[0],indLat.shape[0],indLon.shape[0],1)
y_pred_missing  = y_pred_missing.reshape(y_pred_missing.shape[0],indLat.shape[0],indLon.shape[0],1)
mask_pred       = mask_pred.reshape(mask_pred.shape[0],indLat.shape[0],indLon.shape[0],1)

#Change Nan to 0 Check isany Nan/inf 
def isany(x):
    x[np.where(np.isnan(x))]=0
    print(np.any(np.isnan(x)))
    print(np.any(np.isinf(x)))
    print(x.shape)

isany(x_train)
isany(x_test)
isany(x_train_missing)
isany(x_test_missing)
isany(mask_test)
isany(mask_train)
isany(y_pred)
isany(y_pred_missing)
isany(x_train)
isany(gt_test)
isany(gt_train)
#isany(gt_pred)


print("... (after normalization) mean Tr = %f"%(np.mean(gt_train)))
print("... (after normalization) mean Tt = %f"%(np.mean(gt_test)))
print("... (after normalization) mean x_train = %f"%(np.mean(x_train)))
print("... (after normalization) mean x_train_missing = %f"%(np.mean(x_train_missing)))
print("... (after normalization) mean x_test = %f"%(np.mean(x_test)))
print("... (after normalization) mean x_test_missing = %f"%(np.mean(x_test_missing)))
print("... (after normalization) mean y_pred = %f"%(np.mean(y_pred)))
print("... (after normalization) mean y_pred_missing = %f"%(np.mean(y_pred_missing)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:765: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:706: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


MEAN (4096,)
False
False
(895, 64, 64, 1)
False
False
(441, 64, 64, 1)
False
False
(895, 64, 64, 1)
False
False
(441, 64, 64, 1)
False
False
(441, 64, 64, 1)
False
False
(895, 64, 64, 1)
False
False
(100, 64, 64, 1)
False
False
(100, 64, 64, 1)
False
False
(895, 64, 64, 1)
False
False
(441, 64, 64, 1)
False
False
(895, 64, 64, 1)
... (after normalization) mean Tr = 0.095261
... (after normalization) mean Tt = 0.094365
... (after normalization) mean x_train = 0.095261
... (after normalization) mean x_train_missing = 0.095261
... (after normalization) mean x_test = 0.094365
... (after normalization) mean x_test_missing = 0.014594
... (after normalization) mean y_pred = -0.000000
... (after normalization) mean y_pred_missing = -0.002209


### Define GENN

In [ ]:
# Constraints on kernel to to zeros
# a specific position
class Constraint_Zero(Constraint):
    def __init__(self, position,kernel_shape,dw):
        self.position = position
        mask_array    = np.ones((kernel_shape))
        mask_array[position[0]-dw:position[0]+dw+1,position[1]-dw:position[1]+dw+1,:,:] = 0.0

        self.mask = K.variable(value=mask_array, dtype='float32', name='mask')

        print(self.mask.shape)
    def __call__(self, w):
        print(self.mask.shape)
        new_w = w * self.mask

        return new_w

def fl27(dict_global_Params,x_data,mask_data):

    # import Global Parameters
    for key,val in dict_global_Params.items():
        exec("globals()['"+key+"']=val")

    WFilter       = 11#
    NbResUnit     = 10#3#
    dW            = 0
    flagdownScale = 1 #: 0: only HR scale, 1 : only LR, 2 : HR + LR , 2 : MR, HR + LR annd LR,
    scaleLR       = 2**2
    NbFilter      = 1*DimAE
    flagSRResNet  = 0
     
    input_layer = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))
    mask       = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))
    
    if flagUseMaskinEncoder == 1:
        dmask   = keras.layers.Lambda(lambda x: 0.2*x - 0.1)(mask)
        
        for jj in range(0,6):
            dx    = keras.layers.Conv2D(10,(3,3),activation='relu', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dmask)            
            dx    = keras.layers.Conv2D(1,(3,3),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)            
            dmask = keras.layers.Add()([dmask,dx])
            
        x0       = keras.layers.Concatenate(axis=-1)([input_layer,dmask])
    else:
        x0  = keras.layers.Lambda(lambda x: 1. * x)(input_layer)

    # coarse scale
    if flagdownScale > 0 :
        if flagUseMaskinEncoder == 1:
            x = keras.layers.AveragePooling2D((scaleLR,scaleLR), padding='valid')(x0)
            
            x = keras.layers.Conv2D(NbFilter,(WFilter,WFilter),activation='relu', 
                padding='same',use_bias=False,
                kernel_regularizer=keras.regularizers.l2(wl2),
                kernel_constraint=Constraint_Zero((int(WFilter/2),int(WFilter/2)),(WFilter,WFilter,2*x_train.shape[3],NbFilter),dW))(x)
        else:
            x = keras.layers.AveragePooling2D((scaleLR,scaleLR), padding='valid')(x0)

            x = keras.layers.Conv2D(NbFilter,(WFilter,WFilter),activation='relu', 
            padding='same',use_bias=False,
            kernel_regularizer=keras.regularizers.l2(wl2),
            kernel_constraint=Constraint_Zero((int(WFilter/2),int(WFilter/2)),(WFilter,WFilter,x_train.shape[3],NbFilter),dW))(x)
        x  = keras.layers.Conv2D(DimAE,(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x)
  
        # registration/evolution in feature space
        scale = 0.1
        for kk in range(0,NbResUnit):
            if 1*1 :
                dx = keras.layers.Conv2D(5*DimAE,(1,1),activation='relu', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x)
            else:
                dx = keras.layers.Lambda(lambda x: scale * x)(x)
      
            dx_lin  = keras.layers.Conv2D(DimAE,(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
            dx1 = keras.layers.Conv2D(DimAE,(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
            dx2 = keras.layers.Conv2D(DimAE,(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
      
            dx1 = keras.layers.Multiply()([dx1,dx2])
            
            dx  = keras.layers.Add()([dx1,dx_lin])
            dx  = keras.layers.Activation('tanh')(dx)
            x  = keras.layers.Add()([x,dx])
        x  = keras.layers.Conv2D(int(x_train.shape[3]/(N_cov+1)),(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x)
        x1 = keras.layers.Conv2DTranspose(int(x_train.shape[3]/(N_cov+1)),(scaleLR,scaleLR),strides=(scaleLR,scaleLR),use_bias=False,activation='linear',padding='same',output_padding=None,kernel_regularizer=keras.regularizers.l2(wl2))(x)
        
        if flagSRResNet == 1: ## postprocessing: super-resolution-like block
            x1  = keras.layers.Conv2D(DimAE,(3,3),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x1)               
            
            scale = 0.1
            for kk in range(0,NbResUnit):
                if 1*1 :
                    dx = keras.layers.Conv2D(2*DimAE,(3,3),activation='relu', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x1)
                else:
                    dx = keras.layers.Lambda(lambda x: scale * x)(x1)
          
                dx_lin  = keras.layers.Conv2D(DimAE,(3,3),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
                dx1 = keras.layers.Conv2D(DimAE,(3,3),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
                dx2 = keras.layers.Conv2D(DimAE,(3,3),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
          
                dx1 = keras.layers.Multiply()([dx1,dx2])
        
                dx  = keras.layers.Add()([dx1,dx_lin])
                dx  = keras.layers.Activation('tanh')(dx_lin)
                x1  = keras.layers.Add()([x1,dx])
            x1  = keras.layers.Conv2D(x_train.shape[3]/(N_cov+1),(3,3),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x1)
        else:
            x1  = keras.layers.Conv2D(2*DimAE,(3,3),activation='relu', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x1) 
            x1  = keras.layers.Conv2D(int(x_train.shape[3]/(N_cov+1)),(3,3),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x1)          
     
    # fine scale
    if flagUseMaskinEncoder == 1:
        x = keras.layers.Conv2D(NbFilter,(WFilter,WFilter),activation='relu', 
            padding='same',use_bias=False,
            kernel_regularizer=keras.regularizers.l2(wl2),
            kernel_constraint=Constraint_Zero((int(WFilter/2),int(WFilter/2)),(WFilter,WFilter,2*x_train.shape[3],NbFilter),dW))(x0)
    else:
        x = keras.layers.Conv2D(NbFilter,(WFilter,WFilter),activation='relu', 
            padding='same',use_bias=False,
            kernel_regularizer=keras.regularizers.l2(wl2),
            kernel_constraint=Constraint_Zero((int(WFilter/2),int(WFilter/2)),(WFilter,WFilter,x_train.shape[3],NbFilter),dW))(x0)

    if flagdownScale > 0 :
        x  = keras.layers.Concatenate(axis=-1)([x,x1])
    x  = keras.layers.Conv2D(DimAE,(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x)

    for kk in range(0,NbResUnit):
        if 1*1 :
            dx      = keras.layers.Conv2D(5*DimAE,(1,1),activation='relu', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x)
        else:
            dx  = keras.layers.Lambda(lambda x: scale * x)(x)
          
        dx_lin  = keras.layers.Conv2D(DimAE,(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
        dx1 = keras.layers.Conv2D(DimAE,(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
        dx2 = keras.layers.Conv2D(DimAE,(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(dx)
          
        dx1 = keras.layers.Multiply()([dx1,dx2])
        
        dx  = keras.layers.Add()([dx1,dx_lin])
        dx  = keras.layers.Activation('tanh')(dx)
        x  = keras.layers.Add()([x,dx])

    x  = keras.layers.Conv2D(int(x_train.shape[3]/(N_cov+1)),(1,1),activation='linear', padding='same',use_bias=False,kernel_regularizer=keras.regularizers.l2(wl2))(x)

    if flagdownScale == 0:
        encoder    = keras.models.Model([input_layer,mask],x)
    elif flagdownScale == 1:
        encoder    = keras.models.Model([input_layer,mask],x1)
    elif flagdownScale == 2:
        x          = keras.layers.Add()([x,x1]) 
        encoder    = keras.models.Model([input_layer,mask],x)
    elif flagdownScale == 3:
        x          = keras.layers.Add()([x,x1]) 
        encoder    = keras.models.Model([input_layer,mask],[x1,x])
   
    if flagdownScale == 3:
        decoder_input1 = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))    
        decoder_input2 = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))    
        x1  = keras.layers.Lambda(lambda x: 1. * x)(decoder_input1)
        x2  = keras.layers.Lambda(lambda x: 1. * x)(decoder_input2)
        decoder       = keras.models.Model([decoder_input1,decoder_input2],[x1,x2])
    else:
        decoder_input = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))    
        x  = keras.layers.Lambda(lambda x: 1. * x)(decoder_input)
        decoder       = keras.models.Model(decoder_input,x)
      
    encoder.summary()
    decoder.summary()

    if flagdownScale < 3:
        input_data = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))
        mask       = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))
  
        x          = decoder(encoder([input_data,mask]))
        model_AE   = keras.models.Model([input_data,mask],x)      
    elif flagdownScale == 3:
        
        input_data = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))
        mask       = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))
        x,xLR      = encoder([input_data,mask])
        
        model_AE    = keras.models.Model([input_data,mask],x)
        model_AE_MR = keras.models.Model([input_data,mask],[x,xLR])

    size_tw = int(x_train.shape[3]/(N_cov+1))
    #model_AE.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(lr=1e-3))
    model_AE.compile(loss=keras_custom_loss_function(size_tw),optimizer=keras.optimizers.Adam(lr=1e-3))
    model_AE.summary()

    DimCAE = DimAE

    return encoder, decoder, model_AE, DimCAE

def define_DINConvAE(NiterProjection,model_AE,shape,\
                    size_tw,N_cov=0):
                     

    # encoder-decoder with masked data
    x_input         = keras.layers.Input((shape[1],shape[2],shape[3]))
    mask            = keras.layers.Input((shape[1],shape[2],shape[3]))
    
    x     = keras.layers.Lambda(lambda x:1.*x)(x_input)
    mask_ = keras.layers.Lambda(lambda x:1.-x)(mask)

    # Iterations of fixed-point projection
    index = np.arange(0,(N_cov+1)*size_tw,N_cov+1)   
    for kk in range(0,NiterProjection):
        x_proj   = model_AE([x,mask])
        x_proj   = keras.layers.Multiply()([x_proj,slice_layer(index)(mask_)])
        x        = keras.layers.Multiply()([slice_layer(index)(x),slice_layer(index)(mask)])
        x        = keras.layers.Add()([x,x_proj])

    x_proj = model_AE([x,mask]) 
    global_model_FP    = keras.models.Model([x_input,mask],[x_proj])

    x_input         = keras.layers.Input((shape[1],shape[2],shape[3]))
    mask            = keras.layers.Input((shape[1],shape[2],shape[3]))

    maskg = keras.layers.Lambda(lambda x: 1.*x)(mask)
      
    x_proj = global_model_FP([x_input,maskg])

  
    # AE error with x_proj
    err1 = error(x_proj,x_input,mask,size_tw,shape,1,N_cov)
    # compute error (x_proj-x_input)**2 with full-1 mask
    x_proj2 = model_AE([x_proj,keras.layers.Lambda(lambda x:1.-0.*x)(mask)])
    err2    = error(x_proj,x_proj2,mask,size_tw,shape,0,N_cov)
    # compute error (x_proj-x_input)**2 with full-1 mask
    x_proj3 = model_AE([x_proj,keras.layers.Lambda(lambda x:0.*x)(mask)])
    err3    = error(x_proj3,x_proj,mask,size_tw,shape,0,N_cov)
    # add all errors
    err    = keras.layers.Add()([err1,err2])
    err    = keras.layers.Add()([err,err3])

    # return global model
    global_model_FP_Masked  = keras.models.Model([x_input,mask],err)
    global_model_FP.summary()
    global_model_FP_Masked.summary()
  
    return global_model_FP,global_model_FP_Masked

def slice_layer(index):
    def func(x_input):
        return tf.gather(x_input, index, axis=3)
    return keras.layers.Lambda(func)

def assign_sliced_layer(size_tw,N_cov,x_output):
    def func(x_input,x_output):
        for i in range(1,(N_cov+1)):
            index = np.arange(i,(N_cov+1)*size_tw,(N_cov+1),dtype='int32')
            x_output = keras.layers.Concatenate()([x_output,slice_layer(index)(x_input)])
        index  = np.stack([np.arange(i*size_tw,(i+1)*size_tw) \
                           for i in range(0,N_cov+1)]).T.flatten()
        x_proj = slice_layer(index)(x_output)
        return x_proj
    return keras.layers.Lambda(func,arguments={'x_output':x_output})

def error(x1,x2,mask,size_tw,shape,alpha,N_cov):
    if x1.shape[3]>x2.shape[3]:
        index = np.arange(0,(N_cov+1)*size_tw,N_cov+1,dtype='int32')
        err   = keras.layers.Subtract()([slice_layer(index)(x1),x2])
        err   = keras.layers.Multiply()([err,slice_layer(index)(mask)])
    elif x2.shape[3]>x1.shape[3]:
        index = np.arange(0,(N_cov+1)*size_tw,N_cov+1,dtype='int32')
        err   = keras.layers.Subtract()([x1,slice_layer(index)(x2)])
        err   = keras.layers.Multiply()([err,slice_layer(index)(mask)])
    else:
        err   = keras.layers.Subtract()([x1,x2])
        err   = keras.layers.Multiply()([err,mask])
    err   = keras.layers.Multiply()([err,err])
    err   = keras.layers.Reshape((err.shape[-3],err.shape[-2],err.shape[-1],1))(err)
    err   = keras.layers.GlobalAveragePooling3D()(err)
    err   = keras.layers.Reshape((1,))(err)
    err   = keras.layers.Lambda(lambda x: alpha*x)(err)
    return err

def keras_custom_loss_function(size_tw):
    def insert_Sobel(size_tw,dir="x"):
        kernel_weights=np.zeros((3,3,size_tw,size_tw))
        if dir=="x":
            sobel=np.array([[-1,0,1],[-2,0,2],[-1,0,1]]).T
        if dir=="y":
            sobel=np.array([[-1,-2,-1],[0,0,0],[1,2,1]]).T
        for i in range(size_tw):
            kernel_weights[:,:,i,i]=sobel
        return kernel_weights
    def lossFunction(y_true,y_pred):
        mae = tf.keras.losses.mean_absolute_error(y_true, y_pred)
        filter_gx  = K.constant(insert_Sobel(size_tw,"x"))
        filter_gy  = K.constant(insert_Sobel(size_tw,"y"))
        Gx_true = K.conv2d(y_true, filter_gx, padding="same")
        Gy_true = K.conv2d(y_true, filter_gy, padding="same")
        Grad_true = K.sqrt(keras.layers.Add()([keras.layers.Multiply()([Gx_true,Gx_true]),\
                                       keras.layers.Multiply()([Gy_true,Gy_true])]))
        Gx_pred = K.conv2d(y_pred, filter_gx, padding="same")
        Gy_pred = K.conv2d(y_pred, filter_gy, padding="same")
        Grad_pred = K.sqrt(keras.layers.Add()([keras.layers.Multiply()([Gx_pred,Gx_pred]),\
                                       keras.layers.Multiply()([Gy_pred,Gy_pred])]))
        mae_grad = tf.keras.losses.mean_absolute_error(Grad_true, Grad_pred)
        alpha= 0.5
        loss = ((1.-alpha)*mae) + (alpha*mae_grad)
        return loss
    return lossFunction

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

def eval_AEPerformance(x_train,rec_AE_Tr,x_test,rec_AE_Tt):

    mse_AE_Tr        = np.mean( (rec_AE_Tr - x_train)**2 )
    var_Tr           = np.mean( (x_train-np.mean(x_train,axis=0)) ** 2 )
    exp_var_AE_Tr    = 1. - mse_AE_Tr / var_Tr
   
    mse_AE_Tt        = np.mean( (rec_AE_Tt - x_test)**2 )
    var_Tt           = np.mean( (x_test-np.mean(x_train,axis=0))** 2 )
    exp_var_AE_Tt    = 1. - mse_AE_Tt / var_Tt
           
    return exp_var_AE_Tr,exp_var_AE_Tt

# functions for the evaluation of interpolation and auto-encoding performance
def eval_AEPerformance(x_train,rec_AE_Tr,x_test,rec_AE_Tt):

    mse_AE_Tr        = np.mean( (rec_AE_Tr - x_train)**2 )
    var_Tr           = np.mean( (x_train-np.mean(x_train,axis=0)) ** 2 )
    exp_var_AE_Tr    = 1. - mse_AE_Tr / var_Tr
    
    mse_AE_Tt        = np.mean( (rec_AE_Tt - x_test)**2 )
    var_Tt           = np.mean( (x_test-np.mean(x_train,axis=0))** 2 )
    exp_var_AE_Tt    = 1. - mse_AE_Tt / var_Tt
            
    return exp_var_AE_Tr,exp_var_AE_Tt

def eval_InterpPerformance(mask_train,x_train,x_train_missing,x_train_pred,
                           mask_test,x_test,x_test_missing,x_test_pred):
    mse_train      = np.zeros((2))
    mse_train[0]   = np.sum( mask_train * (x_train_pred - x_train_missing)**2 ) / np.sum( mask_train )
    mse_train[1]   = np.mean( (x_train_pred - x_train)**2 )
    exp_var_train  = 1. - mse_train #/ var_Tr
            
    mse_test        = np.zeros((2))
    mse_test[0]     = np.sum( mask_test * (x_test_pred - x_test_missing)**2 ) / np.sum( mask_test )
    mse_test[1]     = np.mean( (x_test_pred - x_test)**2 ) 
    exp_var_test = 1. - mse_test #/ var_Tt

    mse_train_interp        = np.sum( (1.-mask_train) * (x_train_pred - x_train)**2 ) / np.sum( 1. - mask_train )
    exp_var_train_interp    = 1. - mse_train_interp 
    
    mse_test_interp        = np.sum( (1.-mask_test) * (x_test_pred - x_test)**2 ) / np.sum( 1. - mask_test )
    exp_var_test_interp    = 1. - mse_test_interp
            
    return mse_train,exp_var_train,mse_test,exp_var_test,mse_train_interp,exp_var_train_interp,mse_test_interp,exp_var_test_interp

# function to create recursive paths
def mk_dir_recursive(dir_path):
    if os.path.isdir(dir_path):
        return
    h, t = os.path.split(dir_path)  # head/tail
    if not os.path.isdir(h):
        mk_dir_recursive(h)

    new_path = join_paths(h, t)
    if not os.path.isdir(new_path):
        os.mkdir(new_path)

def Gradient(img, order):
    """ calculate x, y gradient and magnitude """ 
    sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=3)
    sobelx = sobelx/8.0
    sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=3)
    sobely = sobely/8.0
    sobel_norm = np.sqrt(sobelx*sobelx+sobely*sobely)
    if (order==0):
        return sobelx
    elif (order==1):
        return sobely
    else:
        return sobel_norm

def insert_Sobel(size_tw,dir="x"):
    kernel_weights=np.zeros((3,3,size_tw,size_tw))
    if dir=="x":
        sobel=np.array([[-1,0,1],[-2,0,2],[-1,0,1]]).T
    if dir=="y":
        sobel=np.array([[-1,-2,-1],[0,0,0],[1,2,1]]).T
    for i in range(size_tw):
        kernel_weights[:,:,i,i]=sobel
    return kernel_weights

# New loss function for unsupervised setting:
# L = MAE + ||Grad||^2
def keras_custom_loss_function2(size_tw):
    def insert_Sobel(size_tw,dir="x"):
        kernel_weights=np.zeros((3,3,size_tw,size_tw))
        if dir=="x":
            sobel=np.array([[-1,0,1],[-2,0,2],[-1,0,1]]).T
        if dir=="y":
            sobel=np.array([[-1,-2,-1],[0,0,0],[1,2,1]]).T
        for i in range(size_tw):
            kernel_weights[:,:,i,i]=sobel
        return kernel_weights
    def lossFunction(y_true,y_pred):
        filter_gx  = K.constant(insert_Sobel(size_tw,"x"))
        filter_gy  = K.constant(insert_Sobel(size_tw,"y"))
        Gx_pred = K.conv2d(y_pred, filter_gx, padding="same")
        Gy_pred = K.conv2d(y_pred, filter_gy, padding="same")
        Grad_pred = K.sqrt(keras.layers.Add()([keras.layers.Multiply()([Gx_pred,Gx_pred]),\
                                       keras.layers.Multiply()([Gy_pred,Gy_pred])]))
        loss = K.mean(keras.layers.Multiply()([Grad_pred,Grad_pred]))
        return loss
    return lossFunction

def thresholding(x,thr):
    greater = K.greater_equal(x,thr) #will return boolean values
    greater = K.cast(greater, dtype=K.floatx()) #will convert bool to 0 and 1    
    return greater

def slice_layer(index):
    def func(x_input):
        return tf.gather(x_input, index, axis=3)
    return keras.layers.Lambda(func)

def assign_sliced_layer(size_tw,N_cov,x_output):
    def func(x_input,x_output):
        for i in range(1,(N_cov+1)):
            index = np.arange(i,(N_cov+1)*size_tw,(N_cov+1),dtype='int32')
            x_output = keras.layers.Concatenate()([x_output,slice_layer(index)(x_input)])
        index  = np.stack([np.arange(i*size_tw,(i+1)*size_tw) \
                           for i in range(0,N_cov+1)]).T.flatten()
        x_proj = slice_layer(index)(x_output)
        return x_proj
    return keras.layers.Lambda(func,arguments={'x_output':x_output})

def error(x1,x2,mask,size_tw,shape,alpha,N_cov):
    if x1.shape[3]>x2.shape[3]:
        index = np.arange(0,(N_cov+1)*size_tw,N_cov+1,dtype='int32')
        err   = keras.layers.Subtract()([slice_layer(index)(x1),x2])
        err   = keras.layers.Multiply()([err,slice_layer(index)(mask)])
    elif x2.shape[3]>x1.shape[3]:
        index = np.arange(0,(N_cov+1)*size_tw,N_cov+1,dtype='int32')
        err   = keras.layers.Subtract()([x1,slice_layer(index)(x2)])
        err   = keras.layers.Multiply()([err,slice_layer(index)(mask)])
    else:
        err   = keras.layers.Subtract()([x1,x2])
        err   = keras.layers.Multiply()([err,mask])
    err   = keras.layers.Multiply()([err,err])
    err   = keras.layers.Reshape((err.shape[-3],err.shape[-2],err.shape[-1],1))(err)
    # normalize err
    err   = keras.layers.GlobalAveragePooling3D()(err)
    err   = keras.layers.Reshape((1,))(err)
    err   = keras.layers.Lambda(lambda x: alpha*x)(err)
    return err

def regularize_Gradient(x_proj,size_tw):
    filter_gx  = K.constant(insert_Sobel(size_tw,"x"))
    filter_gy  = K.constant(insert_Sobel(size_tw,"y"))
    def Gradient(tensors):
        tensor = tensors[0]
        filter = tensors[1]
        return K.conv2d(tensor, filter, padding="same")
    Gx_pred = keras.layers.Lambda(Gradient)([x_proj, filter_gx])
    Gy_pred = keras.layers.Lambda(Gradient)([x_proj, filter_gy])
    #Grad_pred = K.sqrt(keras.layers.Add()([keras.layers.Multiply()([Gx_pred,Gx_pred]),\
    #                                    keras.layers.Multiply()([Gy_pred,Gy_pred])]))
    Grad_pred = keras.layers.Add()([keras.layers.Multiply()([Gx_pred,Gx_pred]),\
                                        keras.layers.Multiply()([Gy_pred,Gy_pred])])
    reg_Gradient   = keras.layers.Reshape((Grad_pred.shape[-3],Grad_pred.shape[-2],Grad_pred.shape[-1],1))(Grad_pred)
    reg_Gradient   = keras.layers.GlobalAveragePooling3D()(reg_Gradient)
    reg_Gradient   = keras.layers.Reshape((1,))(reg_Gradient)
    reg_Gradient   = keras.layers.Lambda(lambda x: 1*x)(reg_Gradient)
    return reg_Gradient

def define_DINConvAE(NiterProjection,model_AE,shape,\
                     flagUseMaskinEncoder,\
                     size_tw,include_covariates,N_cov=0):

    # encoder-decoder with masked data
    x_input         = keras.layers.Input((shape[1],shape[2],shape[3]))
    mask            = keras.layers.Input((shape[1],shape[2],shape[3]))
    
    x     = keras.layers.Lambda(lambda x:1.*x)(x_input)
    mask_ = keras.layers.Lambda(lambda x:1.-x)(mask)

    # Iterations of fixed-point projection
    index = np.arange(0,(N_cov+1)*size_tw,N_cov+1)   
    for kk in range(0,NiterProjection):
        x_proj   = model_AE([x,mask])
        x_proj   = keras.layers.Multiply()([x_proj,slice_layer(index)(mask_)])
        x        = keras.layers.Multiply()([slice_layer(index)(x),slice_layer(index)(mask)])
        x        = keras.layers.Add()([x,x_proj])


    x_proj = model_AE([x,mask]) 
    global_model_FP    = keras.models.Model([x_input,mask],[x_proj])

    # randomly sample an additionnal missing data mask
    # additive noise + spatial smoothing
    if flagUseMaskinEncoder == 1:
        WAvFilter     = 3
        NIterAvFilter = 3
        thrNoise      = 1.5 * stdMask + 1e-7
        maskg   = keras.layers.GaussianNoise(stdMask)(mask)
        avFilter       = 1./(WAvFilter**3)*np.ones((WAvFilter,WAvFilter,WAvFilter,1,1))
        spatialAvLayer = keras.layers.Conv3D(1,(WAvFilter,WAvFilter,WAvFilter),weights=[avFilter],\
                           padding='same',activation='linear',use_bias=False,name='SpatialAverage')
        spatialAvLayer.trainable = False
        maskg = keras.layers.Lambda(lambda x: K.permute_dimensions(x,(0,3,1,2)))(maskg) 
        maskg  = keras.layers.Reshape((shape[3],shape[1],shape[2],1))(maskg)
        for nn in range(0,NIterAvFilter):
            maskg  = spatialAvLayer(maskg) 
        maskg = keras.layers.Lambda(lambda x: K.permute_dimensions(x,(0,2,3,1,4)))(maskg) 
        maskg = keras.layers.Reshape((shape[1],shape[2],shape[3]))(maskg)
        maskg = keras.layers.Lambda(lambda x: thresholding(x,thrNoise))(maskg)    
        maskg  = keras.layers.Multiply()([mask,maskg])
        maskg  = keras.layers.Subtract()([mask,maskg])       
    else:
        maskg = keras.layers.Lambda(lambda x: 1.*x)(mask)

    x_proj = global_model_FP([x_input,maskg])
    # AE error with x_proj
    err1 = error(x_proj,x_input,mask,size_tw,shape,1,N_cov)
    # AE error with x_proj
    # compute error (x_proj-x_input)**2 with full-1 mask
    x_proj_ = x_proj
    
    x_proj2 = model_AE([x_proj_,keras.layers.Lambda(lambda x:1.-0.*x)(mask)])
    err2    = error(x_proj_,x_proj2,mask,size_tw,shape,0,N_cov)
    # compute error (x_proj-x_input)**2 with full-1 mask
    x_proj3 = model_AE([x_proj_,keras.layers.Lambda(lambda x:0.*x)(mask)])
    err3    = error(x_proj3,x_proj_,mask,size_tw,shape,0,N_cov)
    # add all errors
    err    = keras.layers.Add()([err1,err2])
    err    = keras.layers.Add()([err,err3])

    # return global model
    global_model_FP_Masked  = keras.models.Model([x_input,mask],[err,x_proj])

    global_model_FP.summary()
    global_model_FP_Masked.summary()
    return global_model_FP,global_model_FP_Masked

In [ ]:
#Apply model     
encoder, decoder, model_AE, DimCAE = fl27(globParams,x_train,mask_train)

(11, 11, 1, 20)
(11, 11, 1, 20)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 64, 64, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 16, 16, 1)    0           lambda[0][0]                     
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 20)   2420        average_pooling2d[0][0]          
______________________________________________________________

### Train-Model - FP - Iterated projection

In [ ]:
def flagProcess4_Optim0(dict_global_Params,x_train,x_train_missing,mask_train,gt_train,\
                    x_test,x_test_missing,mask_test,gt_test,lday_test,encoder,decoder,model_AE,DimCAE):

    for key,val in dict_global_Params.items():
        exec("globals()['"+key+"']=val")

    meanTr = stdsc.mean_
    stdTr = np.mean(stdsc.var_)
    # ***************** #
    # model compilation #
    # ***************** #

    # model fit
    NbProjection   = [0,0,2,2,5,5,10,15,14]
    NbProjection   = [5,5,5,5]
    lrUpdate       = [1e-3,1e-4,1e-5,1e-5,1e-5,1e-6,1e-6,1e-5,1e-6]
    if flagTrOuputWOMissingData==0:
        lrUpdate   = [1e-4,1e-5,1e-6,1e-7]
    else:
        lrUpdate   = [1e-3,1e-4,1e-5,1e-6]
    IterUpdate     = [0,3,10,15,20,25,30,35,40]
    #IterUpdate     = [0,6,15,20]
    val_split      = 0.1
    
    iterInit = 0
    IterTrainAE = 0
    IterUpdateInit = 10000
    
    ## initialization
    x_train_init = np.copy(x_train_missing)
    x_test_init  = np.copy(x_test_missing)

    comptUpdate = 0

    # ******************** #
    # Start Learning model #
    # ******************** #
        
    print("..... Start learning AE model")
    for iter in tqdm(range(iterInit,Niter)):
        if iter == IterUpdate[comptUpdate]:
            # update DINConvAE model
            NBProjCurrent = NbProjection[comptUpdate]
            print("..... Update/initialize number of projections in DINCOnvAE model # %d"%(NbProjection[comptUpdate]))
            global_model_FP,global_model_FP_Masked = define_DINConvAE(NbProjection[comptUpdate],model_AE,x_train.shape,\
                                                                          flagUseMaskinEncoder,\
                                                                          size_tw,N_cov)
            if flagTrOuputWOMissingData == 1:
                global_model_FP.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(lr=lrUpdate[comptUpdate]))
                #global_model_FP.compile(loss=keras_custom_loss_function(size_tw),optimizer=keras.optimizers.Adam(lr=lrUpdate[comptUpdate]))
            else:
                #global_model_FP_Masked.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(lr=lrUpdate[comptUpdate]))
                global_model_FP_Masked.compile(loss=['mean_squared_error',keras_custom_loss_function2(size_tw)],loss_weights=[1.0,1e-10],optimizer=keras.optimizers.Adam(lr=lrUpdate[comptUpdate]))
            if comptUpdate < len(NbProjection)-1:
                comptUpdate += 1
        
        # gradient descent iteration            
        if flagTrOuputWOMissingData == 1:
            history = global_model_FP.fit([x_train_init,mask_train],gt_train,
                  batch_size=batch_size,
                  epochs = NbEpoc,
                  verbose = 1, 
                  validation_split=val_split)
        else:
            history = global_model_FP_Masked.fit([x_train_init,mask_train],[np.zeros((x_train_init.shape[0],1)),gt_train],
                  batch_size=batch_size,
                  epochs = NbEpoc,
                  verbose = 1, 
                  validation_split=val_split)

        # *********************** #
        # Prediction on test data #
        # *********************** #

        # trained full-model
        x_train_pred    = global_model_FP.predict([x_train_init,mask_train])
        x_test_pred     = global_model_FP.predict([x_test_init,mask_test])

        # trained AE applied to gap-free data
        if flagUseMaskinEncoder == 1:
          rec_AE_Tr     = model_AE.predict([x_train,np.zeros((mask_train.shape))])
          rec_AE_Tt     = model_AE.predict([x_test,np.zeros((mask_train.shape))])
        else:
          rec_AE_Tr     = model_AE.predict([x_train,np.ones((mask_train.shape))])
          rec_AE_Tt     = model_AE.predict([x_test,np.ones((mask_test.shape))])


        mse_train,exp_var_train,\
        mse_test,exp_var_test,\
        mse_train_interp,exp_var_train_interp,\
        mse_test_interp,exp_var_test_interp =\
        eval_InterpPerformance(mask_train,x_train,x_train_missing,x_train_pred,\
                               mask_test,x_test,x_test_missing,x_test_pred)
        
        # interpolation and reconstruction score for the center image
        # when dealing with time series
        if x_train_init.shape[3] > 0 :
            
            dWCenter    = 32  
            
            dT = np.floor( x_train_init.shape[3] / 2 ).astype(int)
            mse_train_center        = np.mean( (x_train_pred[:,:,:,dT] - x_train[:,:,:,dT] )**2 )
            mse_train_center_interp = np.sum( (x_train_pred[:,:,:,dT]  - x_train[:,:,:,dT] )**2 * (1.-mask_train[:,:,:,dT])  ) / np.sum( (1.-mask_train[:,:,:,dT]) )
            
            mse_test_center         = np.mean( (x_test_pred[:,:,:,dT] - x_test[:,:,:,dT] )**2 )
            mse_test_center_interp  = np.sum( (x_test_pred[:,:,:,dT]  - x_test[:,:,:,dT] )**2 * (1.-mask_test[:,:,:,dT])  ) / np.sum( (1-mask_test[:,:,:,dT]) )
            
            var_train_center        = np.var(  x_train[:,:,:,dT] )
            var_test_center         = np.var(  x_test[:,:,:,dT] )
            
            exp_var_train_center         = 1.0 - mse_train_center / var_train_center
            exp_var_train_interp_center  = 1.0 - mse_train_center_interp / var_train_center
            exp_var_test_center          = 1.0 - mse_test_center  / var_test_center
            exp_var_test_interp_center   = 1.0 - mse_test_center_interp/ var_test_center
        # AE performance of the trained AE applied to gap-free data
        exp_var_AE_Tr,exp_var_AE_Tt = eval_AEPerformance(x_train,rec_AE_Tr,x_test,rec_AE_Tt)
        
        print(".......... Auto-encoder performance when applied to gap-free data")
        print('.... explained variance AE (Tr)  : %.2f%%'%(100.*exp_var_AE_Tr))
        print('.... explained variance AE (Tt)  : %.2f%%'%(100.*exp_var_AE_Tt))
        
        if flagUseMaskinEncoder == 1:
        
            exp_var_AE_Tr,exp_var_AE_Tt = eval_AEPerformance(x_train,rec_AE_Tr,x_test,rec_AE_Tt)
        
            print('.... explained variance AE (Tr) with mask  : %.2f%%'%(100.*exp_var_AE_Tr))
            print('.... explained variance AE (Tt) with mask  : %.2f%%'%(100.*exp_var_AE_Tt))



        # update training data
        if iter > IterUpdateInit:
            # mask = 0(missing data) ; 1(data)
          x_train_init = mask_train * x_train_missing + (1.-mask_train) * x_train_pred
          x_test_init  = mask_test  * x_test_missing  + (1.-mask_test)  * x_test_pred
    return global_model_FP,global_model_FP_Masked
print('ready')



ready


In [ ]:
global_model_FP,global_model_FP_Masked=flagProcess4_Optim0(globParams,x_train,x_train_missing,mask_train,gt_train,x_test,x_test_missing,mask_test,gt_test,lday_test,encoder,decoder,model_AE,DimCAE)

  0%|          | 0/6 [00:00<?, ?it/s]

..... Start learning AE model
..... Update/initialize number of projections in DINCOnvAE model # 5
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
lambda_94 (Lambda)              (None, 64, 64, 1)    0           input_14[0][0]                   
__________________________________________________________________________________________________
lambda_95 (Lambda)              (None, 64, 64, 1)    0           input_15[0][0]            

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in double_scalars
 17%|█▋        | 1/6 [00:36<03:01, 36.28s/it]

.......... Auto-encoder performance when applied to gap-free data
.... explained variance AE (Tr)  : 91.42%
.... explained variance AE (Tt)  : 91.14%
Epoch 1/6
51/51 [==============================] - 4s 71ms/step - loss: 0.0120 - val_loss: 0.0110
Epoch 2/6
51/51 [==============================] - 4s 78ms/step - loss: 0.0107 - val_loss: 0.0110
Epoch 3/6
51/51 [==============================] - 3s 67ms/step - loss: 0.0112 - val_loss: 0.0136
Epoch 4/6
51/51 [==============================] - 4s 71ms/step - loss: 0.0119 - val_loss: 0.0109
Epoch 5/6
51/51 [==============================] - 3s 66ms/step - loss: 0.0106 - val_loss: 0.0111
Epoch 6/6
51/51 [==============================] - 3s 67ms/step - loss: 0.0107 - val_loss: 0.0118


 33%|███▎      | 2/6 [00:59<02:09, 32.30s/it]

...

.......... Auto-encoder performance when applied to gap-free data
.... explained variance AE (Tr)  : 92.23%
.... explained variance AE (Tt)  : 91.91%
Epoch 1/6
51/51 [==============================] - 4s 71ms/step - loss: 0.0103 - val_loss: 0.0105
Epoch 2/6
51/51 [==============================] - 4s 74ms/step - loss: 0.0103 - val_loss: 0.0106
Epoch 3/6
51/51 [==============================] - 4s 75ms/step - loss: 0.0103 - val_loss: 0.0105
Epoch 4/6
51/51 [==============================] - 4s 69ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 5/6
51/51 [==============================] - 3s 66ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 6/6
51/51 [==============================] - 3s 67ms/step - loss: 0.0102 - val_loss: 0.0105


 83%|████████▎ | 5/6 [02:19<00:28, 28.69s/it]

.......... Auto-encoder performance when applied to gap-free data
.... explained variance AE (Tr)  : 92.27%
.... explained variance AE (Tt)  : 91.95%
Epoch 1/6
51/51 [==============================] - 4s 74ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 2/6
51/51 [==============================] - 4s 73ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 3/6
51/51 [==============================] - 3s 67ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 4/6
51/51 [==============================] - 4s 70ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 5/6
51/51 [==============================] - 4s 69ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 6/6
51/51 [==============================] - 3s 66ms/step - loss: 0.0102 - val_loss: 0.0104


100%|██████████| 6/6 [02:42<00:00, 27.17s/it]

.......... Auto-encoder performance when applied to gap-free data
.... explained variance AE (Tr)  : 92.31%
.... explained variance AE (Tt)  : 91.99%


### Prédiction / Interpolation 

In [ ]:
Pred_GM = global_model_FP.predict([y_pred,mask_pred]).reshape(gt_pred.shape[0],gt_pred.shape[1],gt_pred.shape[2])
Pred_AE = model_AE.predict([y_pred,mask_pred]).reshape(gt_pred.shape[0],gt_pred.shape[1],gt_pred.shape[2])
Pred_AE_norm = (Pred_AE.reshape(len(y_pred),-1)+mean_Tr) .reshape(Pred_AE.shape[0],Pred_AE.shape[1],Pred_AE.shape[2])
Pred_GM_norm = (Pred_GM.reshape(len(y_pred),-1)+mean_Tr).reshape(Pred_AE.shape[0],Pred_AE.shape[1],Pred_AE.shape[2])

In [ ]:
Pred_AE_norm[np.where(Pred_AE_norm<np.nanmin(gt_pred))]=np.nanmin(gt_pred)
Pred_GM_norm[np.where(Pred_GM_norm<np.nanmin(gt_pred))]=np.nanmin(gt_pred)

# Some metrics


In [ ]:
#from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import max_error
from sklearn.metrics import median_absolute_error
import datetime as dt
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.nanmean(np.abs((y_true - y_pred) / y_true)) * 100

def RMSE(a,b):
    """ Compute the Root Mean Square Error between 2 n-dimensional vectors. """ 
    return np.sqrt(np.nanmean((a-b)**2))


def Getmetrics(Target,Pred):
    Target_value = Target.reshape(Target.shape)[np.where(~np.isnan(Target))]
    Pred_value   = Pred.reshape(Target.shape)[np.where(~np.isnan(Target))]
    Target_flat  = Target_value.flatten()
    Pred_flat    = Pred_value.flatten()
    Metrics=dict()

    return {'EVS':explained_variance_score(Target_flat,Pred_flat),'RMSE':RMSE(Target_flat,Pred_flat),'NRMSE':RMSE(Target_flat,Pred_flat)/np.nanmean(Target_flat),'MAbsEr%':mean_absolute_percentage_error(Target_flat,Pred_flat),'MaxEr':max_error(Target_flat,Pred_flat),'MAbsEr':median_absolute_error(Target_flat,Pred_flat),'R²':r2_score(Target_flat,Pred_flat)}

MetricGM=Getmetrics(gt_pred,Pred_GM_norm)
MetricAE=Getmetrics(gt_pred,Pred_AE_norm)
print(MetricGM,MetricAE)

{'EVS': 0.9504256427614494, 'RMSE': 0.12629229049962115, 'NRMSE': -0.03392215609313714, 'MAbsEr%': 1.973055102032464, 'MaxEr': 1.5878864467702805, 'MAbsEr': 0.003409719793125987, 'R²': 0.9503258120518892} {'EVS': 0.9657846878409643, 'RMSE': 0.10483878622165772, 'NRMSE': -0.028159736883042567, 'MAbsEr%': 1.6352918187037824, 'MaxEr': 1.600932371020317, 'MAbsEr': 0.005255662966519692, 'R²': 0.9657688834767162}


In [ ]:
np.save('/content/drive/MyDrive/GENN_GM_norm2',Pred_GM_norm)